In [20]:
# 構造のメタデータを保存するDBを作成する

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence
from sqlalchemy.orm import declarative_base
import uuid

path2db = "/Users/y1u0d2/desktop/Lab/data/qe_data/SiO2"

Base = declarative_base()

class Structure(Base):
    __tablename__ = 'structure'
    id = Column(String, primary_key=True)
    original_path = Column(String, unique=True)
    structure_id = Column(Integer)
    structure_name = Column(String)
    calculation_type = Column(String)

# データベースエンジンの作成とテーブルの初期化
engine = create_engine(f'sqlite:///{path2db}/structure.db')
# Base.metadata.create_all(engine)

# データの追加

In [2]:
from glob import glob
import re

def get_mpid_from_path(path):
    match = re.search(r"mp-\d+", path)
    if match:
        result = match.group(0)
    else:
        result = None
    return result

def generate_unique_uuid(session):
    while True:
        new_uuid = str(uuid.uuid4())
        existing = session.query(Structure).filter_by(id=new_uuid).first()
        if not existing:
            return new_uuid

In [8]:
from sqlalchemy.orm import sessionmaker
import os
import shutil

path2scf = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/amorphous/result_sq"
path2atoms = "/Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/atoms"
calculation_type = "scf"
structure_name = "amorphous"

all_scf_dirs = []
all_scf_dirs.extend(glob(f'{path2scf}/scf*'))


Session = sessionmaker(bind=engine)
session = Session()

for scf_dir in all_scf_dirs:
    mpid = get_mpid_from_path(scf_dir)

    # DBへ追加
    unique_id = generate_unique_uuid(session)
    try:
        structure = Structure(
            id=unique_id,
            original_path=scf_dir, 
            structure_id=mpid, 
            structure_name=structure_name,
            calculation_type=calculation_type
        )
        session.add(structure)
        session.commit()

        # create directory and copy files
        path2target = os.path.join(path2atoms, structure.id)
        # copy scf.in, scf.out and atoms.pkl
        os.makedirs(path2target)
        if calculation_type == "scf":
            shutil.copy(os.path.join(scf_dir, "scf.in"), path2target)
            shutil.copy(os.path.join(scf_dir, "scf.out"), path2target)
            shutil.copy(os.path.join(scf_dir, "atoms.pkl"), path2target)
        elif calculation_type == "relax":
            shutil.copy(os.path.join(scf_dir, "atoms.pkl"), path2target)
    except Exception as e:
        print(f"Error: {e}")
        session.rollback()

session.close()